# iGD: an intergrated genomic data source


In [194]:
import os
import struct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, functools, tqdm, PIL
import time
from multiprocess import Pool
import _pickle as pkl
#chr1:  248,956,422+12,151,146-->15,937*16384
#chr2:  242,193,529+12,945,965-->15,573
#chr3:  198,295,559+10,638,715-->12,753
#chr4:  190,214,555+10,165,685-->12,231
#chr5:  181,538,259+ 9,519,995-->11,662
#chr6:  170,805,979+ 9,130,476-->10,983
#chr7:  159,345,973+ 8,613,298-->10,252
#chr8:  145,138,636+ 8,221,520--> 9,361
#chr9:  138,394,717+ 6,590,811--> 8,850
#chr10: 133,797,422+ 7,223,944--> 8,608
#chr11: 135,086,622+ 7,535,370--> 8,705
#chr12: 133,275,309+ 7,228,129--> 8,576
#chr13: 114,364,328+ 5,082,574--> 7,291
#chr14: 107,043,718+ 4,865,950--> 6,831
#chr15: 101,991,189+ 4,515,076--> 6,501
#chr16:  90,338,345+ 5,101,702--> 5,826
#chr17:  83,257,441+ 4,614,972--> 5,364
#chr18:  80,373,285+ 4,035,966--> 5,152
#chr19:  58,617,616+ 3,858,269--> 3,814
#chr20:  64,444,167+ 3,439,621--> 4,144
#chr21:  46,709,983+ 2,049,697--> 2,977
#chr22:  50,818,468+ 2,135,311--> 3,233
#chrX:  156,040,895+ 5,753,881--> 9,876
#chrY:   57,227,415+   211,643--> 3,506
#0. Prepare:
# file/tile name base: blocksize 2**14=16384 bps
fileBase = "bb14"         #14 bits block
nmax = [15937, 15573, 12753, 12231, 11662, 10983, 10252, 9361, 8850, 8608, 8705, 
        8576, 7291, 6831, 6501, 5826, 5364, 5152, 3814, 4144, 2977, 3233, 9876, 3506]
folder = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 
    'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']
gstart = nmax.copy()       #NW without .copy
for i in range(1, 24):
    gstart[i] += gstart[i-1]
gstart.insert(0, 0)
nTiles = gstart[24]
g2ichr = np.zeros(nTiles, dtype='uint8')
for i in range(24):        #convert block index to ichr
    g2ichr[gstart[i]:gstart[i+1]] = i
#[0, 15937, 31510, 44263, 56494, 68156, 79139, 89391, 98752, 107602, 116210, 124915, 133491, 140782, 147613, 
# 154114,159940, 165304, 170456, 174270, 178414, 181391, 184624, 194500, 198006]

In [195]:
#Append genomic objects to igd file
#filename=filebase+chr_+n.igd (n*16384, n>=0) 
#i: index of the genomic object (eg, ChIP_seq data)
def append_index(filebase, igDf):
    #igDF is a pandas data frame
    igDf.to_csv('igdata/igd_index.tsv', mode='a', sep='\t', header=False)
    return

#binary data: 4 times faster than .igd on creating and 10 times faster on retrieval,
#also take 25% less space
def append_igd(filebase, tmpData):
    #open files and apend region data
    for ichr in range(1, 25):
        ichr1=ichr-1
        for m in range(gstart[ichr1], gstart[ichr]):
            file = open('igdata/'+folder[ichr]+'/'+fileBase+'_'+str(m-gstart[ichr1])+'.igd', 'a')
            file.append(tmpData[ichr1][m])
            file.close()
    return

In [196]:
#Create encode_tfbs binary data .igb
def create_igd():   
    #1. Read head info
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/"
    file = open(file_path+"index.txt")
    headInfo = pd.read_csv(file, delimiter='\t')
    file.close()
    headInfo.to_csv('igdata/igd_index.tsv', sep='\t')
    
    #2. Read region data
    file_path += "regions/"
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)
        
    tmpd = []
    for i in range(0,24):
        tmpd.append(np.empty(nmax[i], dtype=object)) #bytearray

    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np): serialization and deserial..
        rc1 = df[1].values #.astype('uint32')
        rc2 = df[2].values #.astype('uint32')
        rc3 = df[4].values #.astype('uint16')
        for m in range(0, len(rchr)):
            if rchr[m] == 'chrX':
                ichr = 22
            elif rchr[m] == 'chrY':
                ichr = 23
            else:
                ichr = int(rchr[m][3:])-1
            for k in range(0, rcnt[m]):
                idx0 = k+ridx[m]
                idx = n1[idx0]
                rec = struct.pack('IIII', i, rc1[idx0], rc2[idx0], rc3[idx0]) 
                for j in range(0,n2[idx0]+1):
                    if tmpd[ichr][idx+j]==None:
                        tmpd[ichr][idx+j] = rec
                    else:
                        tmpd[ichr][idx+j] += rec            

    #save all files
    t0 = time.time()
    for i in range(0, 24):
        for k in range(0, nmax[i]):
            file = open('igdata/'+folder[i]+'/'+fileBase+'_'+str(k)+'.igd', 'wb')
            #tmp = np.array(tmpd[i][k], dtype=('u4, u4, u4, u2')) 
            #np.save(file, tmp)#, allow_pickle=False)    
            if tmpd[i][k]!=None:
                file.write(tmpd[i][k])          
            #pkl.dump(tmpd[i][k], file) #, protocol=2)
            file.close()
    print('t_save=', time.time()-t0)

In [197]:
#Create encode_tfbs binary data .igb: store the whole data in a single file
def create_igd_w():   
    #1. Read head info
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/"
    file = open(file_path+"index.txt")
    headInfo = pd.read_csv(file, delimiter='\t')
    file.close()
    headInfo.to_csv('igdata/igd_index.tsv', sep='\t')
    
    #2. Read region data: read int64 default--int32 should be better
    file_path += "regions/"
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)
    
    count = np.zeros(nTiles, dtype=np.uint32)    
    data = np.empty(nTiles, dtype=object)        #bytearray        
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1 
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np): serialization and deserial..
        rc1 = df[1].values #.astype('uint32')
        rc2 = df[2].values #.astype('uint32')
        rc3 = df[4].values #.astype('uint16')
        #rec_bytes = np.array(rc1, dtype=int)
        for m in range(0, len(rchr)):
            if rchr[m] == 'chrX':
                ichr = 22
            elif rchr[m] == 'chrY':
                ichr = 23
            else:
                ichr = int(rchr[m][3:])-1
            for k in range(0, rcnt[m]):
                idx0 = k+ridx[m]
                idx = n1[idx0]+gstart[ichr]
                #16 bytes for fast pack/unpack
                rec = struct.pack('IIII', i, rc1[idx0], rc2[idx0], rc3[idx0])          
                for j in range(0,n2[idx0]+1):
                    if data[idx+j]==None:
                        data[idx+j] = rec
                    else:
                        data[idx+j] += rec 

    #save all in a single file: each block should have length of x1024 Bytes
    #header: 200,000*(int32 for block starting (x1024), int32 for data length)
    #    total 1550*1024=1587200 Bytes, 198006*8=1584048--pad:788*4=3152 bytes
    t0 = time.time()
    file = open('igdata/'+fileBase+'.igd', 'wb')
    #Write header info: (1587200, count*14)
    for m in range(nTiles):
        if data[m]!=None:
            count[m]=len(data[m]) #in bytes--record length
        else:
            count[m]=0
        
    file.write(count.tostring())
    for m in range(nTiles):
        if count[m]>0:
            file.write(data[m])       
    file.close()
    print('t_save=', time.time()-t0)

In [198]:
#Add files to igd
def add_GObjs(file_path):
    file_ids = next(os.walk(file_path))[2]
    file_ids.sort()
    n_files = len(file_ids)

    tmpd = []
    for m in range(0,24):
        tmpd.append(np.empty(nmax[m], dtype=object))
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = file_path + id_
        regionData = pd.read_csv(file, delimiter='\t', header=None)       
        df = regionData.sort_values(by=[0, 1])   #first by str, then by start
        n1 = df[1].values//16384
        n2 = df[2].values//16384-n1  
        rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)        
        #if a record crosses the block boundary, list it under both blocks (duplicates)
        #the start and end values are kept for fast processing (np): serialization and deserial..
        rc1 = df[1].values #.astype('uint32')
        rc2 = df[2].values #.astype('uint32')
        rc3 = df[4].values #.astype('uint16')
        #rec_bytes = np.array(rc1, dtype=int)
        for m in range(0, len(rchr)):
            if rchr[m] == 'chrX':
                ichr = 22
            elif rchr[m] == 'chrY':
                ichr = 23
            else:
                ichr = int(rchr[m][3:])-1
            for k in range(0, rcnt[m]):
                idx0 = k+ridx[m]
                idx = n1[idx0]
                #rec = [(i, rc1[idx0], rc2[idx0], rc3[idx0])]    #tuple: a region record
                rec = struct.pack('IIII', i, rc1[idx0], rc2[idx0], rc3[idx0]) 
                for j in range(0,n2[idx0]+1):
                    if tmpd[ichr][idx+j]==None:
                        tmpd[ichr][idx+j] = rec
                    else:
                        tmpd[ichr][idx+j] += rec            

    #save all files
    append_igd(fileBase, tmpd)

In [199]:
#Update igd_whole file (assuming it can't be loaded into memory completely):
#load a group of _.igds from sub folders /chr* each time, ... 
def update_igd_w():
    
    return

In [200]:
def test_read():
    #1. test direct access:
    t0 = time.time()
    total = 0
    for m in range(0,24):
        cchr = folder[m]
        for k in range(0, nmax[m]):
            file = open('igdata/'+cchr+'/bb14_'+str(k)+'.igd', 'rb')
            data1 = file.read() 
    dt = time.time()-t0
    print('dt2=', dt)

    #2. test original data:
    t0 = time.time()
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/regions/"
    file_ids = next(os.walk(file_path))[2]
    total = 0
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        file = open(file_path + id_, 'r')        
        df = pd.read_csv(file, delimiter='\t', header=None) 
        #data1 = file.read() #readlines()
        #total += len(data1)
    dt = time.time()-t0
    print('dt3=', dt)

In [201]:
def zip_igd():
    # test zip file:
    #to zip: zip -r igd.zip chr*
    t0 = time.time()
    import io, zipfile
    archive = zipfile.ZipFile('igdata/igd.zip', 'r')
    #print(archive.infolist()[1:100])
    #print(archive.namelist()[100000:100050]) #infolist()
    #time the retrieval process:
    for m in range(0,24):
        cchr = folder[m]
        for k in range(0, nmax[m]):
            file = cchr+'/bb14_'+str(k)+'.igd'
            data1 = archive.read(file)
    archive.close()
    #print(len(data1), data1[0:5])
    dt = time.time()-t0
    print('dt1=', dt)
    
    #test original
    t0 = time.time()
    file_path = "/home/john/LOLA/LOLACore/hg19/encode_tfbs/regions/"
    file_ids = next(os.walk(file_path))[2]
    for i, id_ in tqdm.tqdm(enumerate(file_ids)):
        regionData = pd.read_csv(file_path+id_, delimiter='\t', header=None)        
        #file = open(file_path + id_, 'r')
        #data1 = file.read() #readlines()
    dt = time.time()-t0
    print('dt2=', dt)   

In [202]:
# Get specified block data: igdlist=[(1, 1008), (1, 3890), (6, 1010), (6, 2000), ....]
# return a list of m lists of tuples: should add chr info 
def get_regions(igdlist):
    t0 = time.time()    
    nblocks = len(igdlist)
    tmpd = []
    for m in range(nblocks):
        ichr, k = igdlist[m]
        fname = 'igdata/'+folder[ichr]+'/bb14_'+str(k)+'.igd'
        if os.path.exists(fname):
            file = open(fname, 'rb')
            tmp = list(struct.iter_unpack('IIII', file.read()))
            tmpd.append(tmp)
            file.close()    
    print('time for get_regions: ', time.time()-t0) 
    return tmpd

In [203]:
# Get specified block data: igdlist=[(1, 1008), (1, 3890), (6, 1010), (6, 2000), ....]
def get_regions_w(igdlist):
    t0 = time.time()    
    nblocks = len(igdlist)
    tmpd = []
    for m in range(nblocks):
        ichr, k = igdlist[m]
        file = open('igdata/'+folder[ichr]+'/bb14_'+str(k)+'.igd', 'rb')
        tmp = list(struct.iter_unpack('IIII', file.read()))
        tmpd.append(tmp)
        file.close()    
    print('time for get_regions: ', time.time()-t0) 
    return tmpd

In [204]:
# Get the entire sets
def get_allRegionSets():
    tmpd = []
    for m in range(0,24):
        tmpd.append(np.empty(nmax[m], dtype=object))

    t0 = time.time()
    for m in range(0,24):
        cchr = folder[m]
        for k in range(0, nmax[m]):
            file = open('igdata/'+cchr+'/bb14_'+str(k)+'.igd', 'rb')
            tmpd[m][k] = list(struct.iter_unpack('IIII', file.read()))
            file.close()
    dt0 = time.time()-t0
    
    print(dt0)
    return tmpd

In [205]:
# Get the entire sets
def get_allRegionSets_w():   
    t0 = time.time()

    file = open('igdata/' + fileBase + '.igd', 'rb')
    data = file.read()
    file.close()
    
    #read head:
    i = nTiles*4
    count = list(struct.unpack('I'*nTiles, data[0:i]))
    #igdata = struct.unpack('IIIH'*nRecords, data[i:]) #NW: due to alighment    
    igdata = list(struct.iter_unpack('IIII', data[i:]))   
    file.close()
            
    dt0 = time.time()-t0    
    print(dt0)
    return igdata

In [206]:
#build query set list from bed file: each list of query<-->each igdlist (block)
from operator import itemgetter
def get_igdlist(file_path):
    regionData = pd.read_csv(file_path, delimiter='\t', header=None)
    #regionData.info()
    df = regionData.sort_values(by=[0, 1])   #df[i]--ith column not row
    df.reset_index(drop=True, inplace=True)  #df normally keeps the index!
    n1 = df[1].values//16384
    n2 = df[2].values//16384-n1  
    rchr, ridx, rcnt = np.unique(df[0].values, return_index=True, return_counts=True)   
    igdlist = []   
    for m in range(len(rchr)):
        if len(rchr[m])<6:
            if rchr[m] == 'chrX':
                ichr = 22
            elif rchr[m] == 'chrY':
                ichr = 23
            else:
                ichr = int(rchr[m][3:])-1
            for k in range(rcnt[m]):
                idx0 = k+ridx[m]
                idx = n1[idx0] + gstart[ichr]  #to be sorted uniquely
                for j in range(0,n2[idx0]+1):
                    igdlist.append((idx+j,df[1][idx0], df[2][idx0])) 
    igdlist.sort(key=itemgetter(0))
    igdlist = np.asarray(igdlist, dtype='uint32')
    return igdlist #sort

In [207]:
#directly examine each block, attach chr info to the result (add tuple item +(100,))
def get_overlaps(igdlist):
    t0 = time.time()  
    rblk, ridx, rcnt = np.unique(igdlist[:,0], return_index=True, return_counts=True)  
    nblocks = len(rblk)    
    overlaps = []
    for m in range(nblocks):
        bk = rblk[m]
        if bk<198006:
            ichr = int(g2ichr[bk])
            k = bk - gstart[ichr]   
            fname = 'igdata/'+folder[ichr]+'/bb14_'+str(k)+'.igd'
            if os.path.exists(fname):
                file = open(fname, 'rb')
                regiondb = list(struct.iter_unpack('IIII', file.read()))
                #make it a np array
                file.close() 
                #--find overlaps in this block
                for n in range(rcnt[m]):
                    idx0 = ridx[m]+n
                    q, q1, q2 = igdlist[idx0]
                    for item in regiondb:   #list of tuples (234,52312312,52312612,156), (256,52307985,52308160,590)
                        if not (q2<item[1] or q1>item[2]):
                            overlaps.append(item+(ichr,))

    print('nBlocks,', nblocks)
    print('time for get_overlaps:', time.time()-t0)
    return overlaps

In [208]:
#directly examine each block, attach chr info to the result (add tuple item +(100,))
#using np array in the block
def get_overlaps_w(igdlist):
    t0 = time.time()     
    rblk, ridx, rcnt = np.unique(igdlist[:,0], return_index=True, return_counts=True)  
    nblocks = len(rblk)     
    file  = open('igdata/' + fileBase + '.igd', 'rb')
    len0 = nTiles*4
    count = list(struct.unpack('I'*nTiles, file.read(len0)))
    #print(count[:100])
    mloc = count.copy()
    #[x*16 for x in mloc]
    mloc.insert(0,len0)
    for m in range(1, nTiles):
        mloc[m] += mloc[m-1] 
    #-----------------------------------------------------------  
    overlaps = []   
    for m in range(nblocks):
        bk = rblk[m]
        if bk<nTiles and count[bk]>0:
            ichr = int(g2ichr[bk])   
            file.seek(mloc[bk])
            regiondb = list(struct.iter_unpack('IIII', file.read(count[bk]))) 
            #print('nrec:', len(regiondb)) 
            #--find overlaps in this block
            for n in range(rcnt[m]):
                idx0 = ridx[m]+n
                q, q1, q2 = igdlist[idx0]                    
                for item in regiondb:   #list of tuples (234,52312312,52312612,156)
                    if not (q2<item[1] or q1>item[2]):
                        overlaps.append(item+(ichr,))
                        #print(q1, q2, item[1], item[2])
    #-----------------------------------------------------------
    file.close()
    print('nBlocks,', nblocks)
    print('time for get_overlaps:', time.time()-t0)
    return overlaps

In [209]:
#0. Create igd:
#create_igd()
igdata = get_allRegionSets()
#igdata[0][1]
#create_igd_w()
igdata = get_allRegionSets_w()
len(igdata)

11.3692946434021
2.575859308242798


13449179

In [212]:
#0. Create igd:
#create_igd()
#1. Read a query region set from file
file = "/home/john/LOLA/lola_vignette_data/setC_100.bed" #test.bed"#
igdlist = get_igdlist(file)
for i in range(10):
    print(i, igdlist[i])
overlaps = get_overlaps(igdlist)
#overlaps = get_overlaps_w(igdlist)
overlaps = sorted(overlaps, key=itemgetter(0, 4, 1))
unilaps = pd.DataFrame(overlaps)[1]
unilaps = unilaps.unique()

0 [    1588 26021603 26022934]
1 [    5551 90953248 90954840]
2 [    5742 94090603 94091951]
3 [     6211 101767799 101769320]
4 [     6821 111770653 111771960]
5 [     6822 111770653 111771960]
6 [     8986 147242999 147244124]
7 [     8987 147242999 147244124]
8 [     9886 161972829 161974473]
9 [    10555 172938552 172939569]
nBlocks, 110
time for get_overlaps: 0.04926609992980957


In [213]:
print(len(igdlist))
print(len(overlaps))
print(len(unilaps))
for i in range(1,10):
    print(i, overlaps[i])

110
2778
2455
1 (2, 35576437, 35576797, 141, 10)
2 (2, 45374400, 45374760, 339, 22)
3 (5, 26021708, 26021978, 186, 0)
4 (5, 35576437, 35576707, 268, 10)
5 (5, 45374459, 45374729, 281, 22)
6 (9, 26021714, 26021936, 553, 0)
7 (9, 35576303, 35576727, 454, 10)
8 (9, 35576503, 35576927, 153, 10)
9 (9, 15312174, 15312598, 341, 16)
